# Use Case B - Query builder 
***

### Revised (January 2022) to read all data from the database

### Adjusted (June 2022) to read all inputs from the Knowledge Database

### Using 2-grams, 3-grams, 4-grams with content from the SE Glossary articles, the Statistics Explained articles and OECD's Glossary of Statistical Terms: https://stats.oecd.org/glossary/¶

### Installations instructions

This is a Google Colab notebook. You must have a Google account. 

Launch the notebook and put your own credentials in the chunk with title "Connect to the Virtuoso database"     


### Installations

In [1]:
!pip install SPARQLWrapper
!pip install sparql_dataframe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 6.5 MB/s 
     |████████████████████████████████| 41 kB 601 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install nltk==3.4 ## needs latest version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 4.3 MB/s 
  Created wheel for nltk: filename=nltk-3.4-py3-none-any.whl size=1436396 sha256=e552d75764a04669116999f6d5eb76de74ec9731aa79ed1db62cf2169f2c05e7
  Stored in directory: /root/.cache/pip/wheels/13/b8/81/2349be11dd144dc7b68ab983b58cd2fae353cdc50bbdeb09d0
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [3]:
!apt-get install virtuoso-opensource

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libvirtodbc0 virtuoso-opensource-6.1 virtuoso-opensource-6.1-bin
  virtuoso-opensource-6.1-common virtuoso-server virtuoso-vad-conductor
  virtuoso-vsp-startpage
Suggested packages:
  virtuoso-vad-doc virtuoso-vad-demo virtuoso-vad-tutorial
  virtuoso-vad-rdfmappers virtuoso-vad-sparqldemo virtuoso-vad-syncml
  virtuoso-vad-bpel virtuoso-vad-isparql virtuoso-vad-ods virtuoso-vad-dbpedia
  virtuoso-vad-facetedbrowser
The following NEW packages will be installed:
  libvirtodbc0 virtuoso-opensource virtuoso-opensource-6.1
  virtuoso-opensource-6.1-bin virtuoso-opensource-6.1-common virtuoso-server
  virtuoso-vad-conductor virtuoso-vsp-startpage
0 upgraded, 8 newly installed, 0 to remove and 20 not upgraded.


In [4]:
## no more needed
## !pip install "ipywidgets>=7,<8" ## temporary fix, see https://github.com/googlecolab/colabtools/issues/3020


### Imports and settings

In [5]:
import pandas as pd
import numpy as np

import re
import unicodedata as ud

from SPARQLWrapper import SPARQLWrapper, POST, DIGEST, GET
from SPARQLWrapper import JSON, INSERT, DELETE
import sparql_dataframe

import gensim

pd.set_option('display.max_colwidth', 40)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


### Connect to the Virtuoso database

In [6]:
user = 'kimon'
passw = 'RkhvQYZ442e2JVXLHdtW'

In [7]:
def connect_virtuoso(DSN, UID, PWD):

    sparql = SPARQLWrapper(DSN)
    sparql.setHTTPAuth(DIGEST)
    sparql.setCredentials(UID, PWD)
    sparql.setMethod(GET)

    return sparql

# Connection to the KDB 
endpoint = "http://lod.csd.auth.gr:8890/sparql/"
sparql = connect_virtuoso(endpoint,user,passw)

###The data cleansing function

In [8]:

def clean(x, quotes=True):
    if pd.isnull(x): return x  
    x = x.strip()
    
    ## make letter-question mark-letter -> letter-quote-space-letter !!! but NOT in the lists of URLs!!!
    if quotes:
        x = re.sub(r'([A-Za-z])\?([A-Za-z])','\\1\' \\2',x) 
    
    ## make letter-question mark-space lower case letter letter-quote-space letter
    x = re.sub(r'([A-Za-z])\? ([a-z])','\\1\' \\2',x) 

    ## delete ,000 commas in numbers    
    x = re.sub(r'\b(\d+),(\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## delete  000 spaces in numbers
    x = re.sub(r'\b(\d+) (\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## remove more than one spaces
    x = re.sub(r' +', ' ',x)
    
    ## remove start and end spaces
    x = re.sub(r'^ +| +$', '',x,flags=re.MULTILINE) 
    
    ## space-comma -> comma
    x = re.sub(r' \,',',',x)
    
    ## space-dot -> dot
    x = re.sub(r' \.','.',x)
    
    #x = x.encode('latin1').decode('utf-8') ## â\x80\x99
    x = ud.normalize('NFKD',x).encode('ascii', 'ignore').decode()
    
    return x

### Glossary articles  

* Titles, URLs and definitions.
* Exclude some redirections.
* Exclude one invalid URL from a remnant empty page.

### The SPARQL query

In [9]:
RelationsStatements = """
DEFINE input:inference <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/>
PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
PREFIX estatdata: <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/>
select ?a ?title ?url ?content where {
    ?a a estat:GlossaryArticle .
    ?a estat:title ?title .
    ?a estat:hasURL ?url.
    ?a estat:content ?content .
    filter(!regex(?content, "^(The revision|Redirect to)")) 
    filter( regex(?url, "Glossary:")) 
} 
"""
## estat:GlossaryArticle OR StatisticsExplainedArticle
  
sparql.setQuery(RelationsStatements)
sparql.method = "POST"
sparql.setReturnFormat(JSON)
GL_df = sparql.query().convert()['results']['bindings']
GL_df = pd.json_normalize(GL_df)
print(GL_df.columns)


Index(['a.type', 'a.value', 'title.type', 'title.value', 'url.type',
       'url.datatype', 'url.value', 'content.type', 'content.value'],
      dtype='object')


### Processing and cleansing

In [10]:
GL_df.rename(columns={'title.value':'title','url.value':'url','content.value':'definition'},inplace=True)
GL_df['id'] = range(len(GL_df))
GL_df = GL_df[['id','title','url','definition']]

GL_df= GL_df.replace('', np.nan) ## Check for anything empty
print(GL_df.isnull().sum())

## Check for duplicates
idx = GL_df[GL_df.duplicated(subset=['title','definition'], keep=False)].sort_values(by=['title','definition']).index
print(idx)

GL_df['title'] = GL_df['title'].apply(clean)
GL_df['title'] = GL_df['title'].apply(lambda x: re.sub(r'\?','-',x)) ## also replace question marks by dashes
GL_df['definition'] = GL_df['definition'].apply(clean)
GL_df['url'] = GL_df['url'].apply(clean,quotes=False)  
GL_df['url'] = GL_df['url'].apply(lambda x: [x]) ## also put each URL in a list - required later
GL_df

id            0
title         0
url           0
definition    0
dtype: int64
Int64Index([], dtype='int64')


,id,title,url,definition
0,0,Accident at work,[https://ec.europa.eu/eurostat/stati...,An accident at work in the framework...
1,1,Gross domestic product GDP,[https://ec.europa.eu/eurostat/stati...,Gross domestic product abbreviated a...
2,2,Toxicity,[https://ec.europa.eu/eurostat/stati...,Toxicity measures the degree to whic...
3,3,Structural fund,[https://ec.europa.eu/eurostat/stati...,The Structural funds are funding ins...
4,4,PRODCOM,[https://ec.europa.eu/eurostat/stati...,PRODCOM is an annual survey for the ...
...,...,...,...,...
1276,1276,Gross operating rate,[https://ec.europa.eu/eurostat/stati...,The gross operating rate in structur...
1277,1277,Gross operating surplus,[https://ec.europa.eu/eurostat/stati...,Gross operating surplus or profits i...
1278,1278,Personnel costs,[https://ec.europa.eu/eurostat/stati...,Within the context of structural bus...
1279,1279,Stratum,[https://ec.europa.eu/eurostat/stati...,In statistics a stratum plural strat...


### Statistics explained articles

* Titles, URLs and concatenated content from all paragraphs (excluding abstracts).

### The SPARQL query

In [11]:
RelationsStatements = """
DEFINE input:inference <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/>
PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
PREFIX estatdata: <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/>
select ?x ?title (group_Concat(?para," ") as ?text) (sample(?url) as ?url) where {
    ?x a estat:StatisticsExplainedArticle .
    ?x estat:title ?title .
    ?x estat:hasURL ?url.
    ?x estat:hasParagraph  ?h .
    ?h estat:title ?t1 .
    FILTER(?t1!="Abstract") .
    ?h estat:content ?c .
    bind (concat(?t1," ",?c," ") as ?para)
} group by ?x ?title
"""
  
sparql.setQuery(RelationsStatements)
sparql.method = "POST"
sparql.setReturnFormat(JSON)
SE_df = sparql.query().convert()['results']['bindings']
SE_df = pd.json_normalize(SE_df)
print(SE_df.columns)


Index(['x.type', 'x.value', 'title.type', 'title.value', 'text.type',
       'text.value', 'url.type', 'url.datatype', 'url.value'],
      dtype='object')


### Processing and cleansing

In [12]:
SE_df.rename(columns={'title.value':'title','url.value':'url','text.value':'raw content'},inplace=True)
SE_df['id'] = range(len(SE_df))
SE_df = SE_df[['id','title','url','raw content']].copy()

SE_df['title'] = SE_df['title'].apply(clean)
SE_df['title'] = SE_df['title'].apply(lambda x: re.sub(r'\?','-',x)) ## also replace question marks by dashes
SE_df['url'] = SE_df['url'].apply(clean,quotes=False)
SE_df['url'] = SE_df['url'].apply(lambda x: [x]) ## put in list
SE_df['raw content'] = SE_df['raw content'].apply(clean)
#SE_df.to_excel('SE_df.xlsx')
SE_df

,id,title,url,raw content
0,0,Russia EU international trade in goo...,[https://ec.europa.eu/eurostat/stati...,Recent developments impact of COVID ...
1,1,Africa EU key statistical indicators,[https://ec.europa.eu/eurostat/stati...,Population and health With the world...
2,2,Comparative price levels for investment,[https://ec.europa.eu/eurostat/stati...,Overview In 2020 the highest price l...
3,3,First and second generation immigran...,[https://ec.europa.eu/eurostat/stati...,General overview The EU attracts qui...
4,4,Migrant integration statistics intro...,[https://ec.europa.eu/eurostat/stati...,Migrant integration in the EU The co...
...,...,...,...,...
860,860,EU statistics on income and living c...,[https://ec.europa.eu/eurostat/stati...,Description The indicator on transit...
861,861,European Neighbourhood Policy South ...,[https://ec.europa.eu/eurostat/stati...,Gross value added and employment Bet...
862,862,EU statistics on income and living c...,[https://ec.europa.eu/eurostat/stati...,Description Each one of the indicato...
863,863,E commerce statistics,[https://ec.europa.eu/eurostat/stati...,E sales record a slight increase ove...


### Read OECD's terms, conexts and definitions
* Terms, URLs, definitions and contexts.
* Also put the URLs in lists. This is required later.

In [13]:
RelationsStatements = """
DEFINE input:inference <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/>
PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
PREFIX estatdata: <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/>
select ?a ?title ?url ?context ?definition where{
    ?a a estat:OECDTerm .
    ?a estat:term ?title .
    ?a estat:hasURL ?url .
    optional { ?a estat:context ?context }
    ?a estat:definition ?definition .
}
"""
  
sparql.setQuery(RelationsStatements)
sparql.method = "POST"
sparql.setReturnFormat(JSON)
OECD_df = sparql.query().convert()['results']['bindings']
OECD_df = pd.json_normalize(OECD_df)
print(OECD_df.columns)


Index(['a.type', 'a.value', 'title.type', 'title.value', 'url.type',
       'url.datatype', 'url.value', 'context.type', 'context.value',
       'definition.type', 'definition.value'],
      dtype='object')


### Processing and cleansing

In [14]:
OECD_df.rename(columns={'title.value':'term','url.value':'url','definition.value':'definition','context.value':'context'},inplace=True)
OECD_df['id'] = range(len(OECD_df))

OECD_df = OECD_df[['id','term','url','definition','context']].copy()


OECD_df.replace('',np.nan,inplace=True)

print(OECD_df.isnull().sum()) ## check for missing values

OECD_df['term'] = OECD_df['term'].apply(clean)
OECD_df['definition'] = OECD_df['definition'].apply(clean)
OECD_df['context'] = OECD_df['context'].apply(clean)
OECD_df['url'] = OECD_df['url'].apply(clean,quotes=False)
OECD_df['url'] = OECD_df['url'].apply(lambda x: [x]) ## also put each URL in a list - required later
OECD_df

id               0
term             0
url              0
definition       0
context       5544
dtype: int64


,id,term,url,definition,context
0,0,Abatement cost,[https://stats.oecd.org/glossary/det...,Abatement costs refer to expenditure...,The calculation of imputed abatement...
1,1,Abnormal obsolescence,[https://stats.oecd.org/glossary/det...,Abnormal obsolescence is the loss in...,Abnormal obsolescence may occur beca...
2,2,Abuse of dominant position,[https://stats.oecd.org/glossary/det...,The term abuse of dominant position ...,These business practices by the firm...
3,3,Accessibility as a statistical data ...,[https://stats.oecd.org/glossary/det...,The ease and the conditions with whi...,Accessibility refers to the availabi...
4,4,Accountability in management theory,[https://stats.oecd.org/glossary/det...,A key concept in modern management t...,The person or body to which the mana...
...,...,...,...,...,...
6940,6940,Zero sum game,[https://stats.oecd.org/glossary/det...,A game played by a number of persons...,NaN
6941,6941,Zero tillage,[https://stats.oecd.org/glossary/det...,Refer No tillage,NaN
6942,6942,Zones,[https://stats.oecd.org/glossary/det...,See Main Economic Indicator main cou...,NaN
6943,6943,Zoning,[https://stats.oecd.org/glossary/det...,Zoning is the process in physical pl...,NaN


### Tokenize, remove stop-words and stem; keep also the original terms

* Use titles and definitions from the Glossary articles.
* Use titles and raw content from the SE articles
* Use terms, definitions and contexts from OECD's glossary entries.
* _texts_ is a list containing lists. Each sub-list has the stemmed term, the original term and the URL where the term was found. The URL is itself put in a list.

In [15]:
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import STOPWORDS

all_stopwords_gensim = STOPWORDS


p = PorterStemmer()

def text_to_words(text,url):
    words = str(gensim.utils.simple_preprocess(text, deacc=True))
    words = gensim.utils.tokenize(words)
    words = [word for word in words if not word in all_stopwords_gensim]    
        
    ## keep also original token!!! 
    words = [[p.stem(token),token,url] for token in words if len(p.stem(token)) >= 5] ##minimum length = 5 
    yield words        

texts=list()   

for i in range(len(GL_df)):
    texts.extend(text_to_words(GL_df.loc[i,'definition'],GL_df.loc[i,'url']))
    texts.extend(text_to_words(GL_df.loc[i,'title'],GL_df.loc[i,'url'])) 
for i in range(len(SE_df)):    
    texts.extend(text_to_words(SE_df.loc[i,'title'],SE_df.loc[i,'url'])) 
    texts.extend(text_to_words(SE_df.loc[i,'raw content'],SE_df.loc[i,'url'])) 
for i in range(len(OECD_df)):    
    texts.extend(text_to_words(OECD_df.loc[i,'term'],OECD_df.loc[i,'url'])) 
    texts.extend(text_to_words(OECD_df.loc[i,'definition'],OECD_df.loc[i,'url'])) 
    OECD_rec = OECD_df.loc[i,'context']
    if not pd.isna(OECD_rec):
        texts.extend(text_to_words(OECD_rec,OECD_df.loc[i,'url'])) 


* Example: the first 5 sub-lists in the first list in _texts_.

In [16]:
print(len(texts))
texts[0][:5]

19583


[['accid',
  'accident',
  ['https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Accident_at_work']],
 ['framework',
  'framework',
  ['https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Accident_at_work']],
 ['administr',
  'administrative',
  ['https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Accident_at_work']],
 ['collect',
  'collection',
  ['https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Accident_at_work']],
 ['european',
  'european',
  ['https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Accident_at_work']]]

### Co-occurences: keys in n-grams are (n-1) tuples of stemmed tokens 

* Three dictionaries, for 2-,3-,and 4-grams. The corresponding keys are single stemmed terms, pairs of stemmed terms and triplets, respectively.
* For each key in a dictionary, the value is another (nested) dictionary with the **original terms**, their counts and the relevant URLs. In the end, the counts are used to calculate probabilities.
* Below all three dictionaries are constructed from the sequences of 4-grams.


In [17]:

from nltk import bigrams, trigrams, ngrams ## ngrams needs version 3.4
#from collections import Counter, defaultdict


model2=dict()
model3=dict()
model4=dict()

def dict_insert(model,entered,proposed,new_urls_to_check):
    key_1 = model.get(entered)
    if key_1:
        key_2 = key_1.get(proposed)
        if key_2:
            key_2[0] +=1
            existing_urls = key_2[1]
            add_urls = [u for u in new_urls_to_check if u not in existing_urls]
            #key_2.extend(add_urls) 
            key_2[1].extend(add_urls) 

        else:    
            key_1[proposed]= [1,new_urls_to_check]
    else:
        model[entered]={proposed:[1,new_urls_to_check]}
    return model            


# Co-occurences

for sentence in texts:
    pairs = [elem for elem in sentence] ## a list of 3-tuples (stemmed term, original term, list of URLs)
    if len(pairs) == 0: continue

    for first, second, third, fourth in ngrams(pairs,4): ## quadruplets of 3-tuples (stemmed term, original term, list of URLs)
        first_stem, first_orig, first_url = first
        second_stem, second_orig, second_url = second
        third_stem, third_orig, third_url = third
        fourth_stem, fourth_orig, fourth_url = fourth
        model2 = dict_insert(model2, first_stem, second_orig,list(set(first_url).intersection(second_url)))
        model2 = dict_insert(model2, second_stem, third_orig,list(set(second_url).intersection(third_url)))
        model2 = dict_insert(model2, third_stem, fourth_orig,list(set(third_url).intersection(fourth_url)))
        
        model3 = dict_insert(model3,(first_stem,second_stem),third_orig,list(set(first_url).intersection(*[second_url,third_url])))
        model3 = dict_insert(model3,(second_stem,third_stem),fourth_orig,list(set(second_url).intersection(*[third_url,fourth_url])))
        
        ##model4 = dict_insert(model4,(first_stem, second_stem, third_stem),fourth_orig,fourth_url)
        model4 = dict_insert(model4,(first_stem, second_stem, third_stem),fourth_orig,list(set(first_url).intersection(*[second_url,third_url,fourth_url])))
   
        
## Transform counts to probabilities

for w1 in model2.keys():
    ssum = sum(model2[w1][w2][0] for w2 in model2[w1].keys())
    for w2 in model2[w1].keys():
        model2[w1][w2][0] /= ssum

for w1_w2 in model3.keys():
    ssum = sum(model3[w1_w2][w3][0] for w3 in model3[w1_w2].keys())
    for w3 in model3[w1_w2].keys():
        model3[w1_w2][w3][0] /= ssum

for w1_w2_w3 in model4.keys():
    ssum = sum(model4[w1_w2_w3][w4][0] for w4 in model4[w1_w2_w3].keys())
    for w4 in model4[w1_w2_w3].keys():
        model4[w1_w2_w3][w4][0] /= ssum


### Examples of keys and values in the three dictionaries

* We do not show an example of the 2-grams dictionary because the values are too many.


In [18]:

print('\nExample, model with 3-grams, key = (household,expenditur):\n')
print(model3[('household','expenditur')])

print('\nExample, model with 4-grams, key = (survei,structur,agricultur):\n')
print(model4[(('survei','structur','agricultur'))])



Example, model with 3-grams, key = (household,expenditur):

{'services': [0.06, ['https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Household_budget_survey_(HBS)', 'https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Ageing_Europe_-_statistics_on_pensions,_income_and_expenditure']], 'order': [0.02, ['https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Building_the_System_of_National_Accounts_-_strategy']], 'consumer': [0.02, ['https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Building_the_System_of_National_Accounts_-_strategy']], 'average': [0.02, ['https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Interaction_of_household_income,_consumption_and_wealth_%E2%80%93_statistics_on_taxation']], 'confirms': [0.02, ['https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Interaction_of_household_income,_consumption_and_wealth_%E2%80%93_statistics_on_taxation']], 'respectively': [0.08, ['htt

### The widgets
***

In [19]:
import ipywidgets as widgets
layout = widgets.Layout(width='600px', height='30px')

In [20]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [21]:
# Avoids scroll-in-the-scroll in the entire Notebook
from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

In [22]:

def change_top_articles( Keywords, Glossaries):

    from operator import itemgetter
    p = PorterStemmer()
    
    last_match = ''
    
    #if not Keywords.endswith(' '):
    #    return None
    
    def test_and_back_step(x):
        mod_index = -1
        models = [model2,model3,model4]
        if len(x)==1: 
            x=x[0] ; model=models[0]
        elif len(x) ==2 or len(x) == 3:
            x=tuple(x) ; mod_index=len(x)-1; model=models[mod_index]
        elif len(x) >=4:
            x=tuple(x[:3]) ; model=models[2]
#        else:
#            return None
        while not models[mod_index].get(x) and mod_index >=0:
            x=x[:-1]
            if len(x)==1 : x=x[0]
            mod_index -=1 ; model=models[mod_index] 
        return (model,x)    
            

    x = Keywords.split() 
    if len(x) ==0: 
        print()
        return
    x = [p.stem(el) for el in x]
    
    
    model,x = test_and_back_step(x)
    if not model.get(x):
        return None
    

    print()
    print('Based on last match: ',x,'\n')
    print('Suggestions, probabilities (in descending order) and relevant URLs: ')
    proposals = sorted([(k,v) for (k,v) in model[x].items()],key=itemgetter(1),reverse=True)
    
    ## Adjust sum of probabilities
    if Glossaries == 'OECD':
        valid_urls = [(v[0],list(filter(lambda x: re.match(r'^https://stats.oecd.org/',x),v[1]))) for (k,v) in proposals]
        valid_urls_sum = sum([v for (v,l) in valid_urls if len(l)>0])
    elif Glossaries == 'Eurostat':    
        valid_urls = [(v[0],list(filter(lambda x: re.match(r'^https://ec.europa.eu/eurostat/',x),v[1]))) for (k,v) in proposals]
        valid_urls_sum = sum([v for (v,l) in valid_urls if len(l)>0])
    else:
        valid_urls_sum = 1.0
    #print('valid_sum: ',valid_urls_sum)

    last_match = x
    sump = 0
    for key, value in proposals:
        urls = value[1]
        if Glossaries == 'OECD':
            urls = [url for url in urls if re.match(r'^https://stats.oecd.org/',url)]
        elif Glossaries == 'Eurostat':   
            urls = [url for url in urls if re.match(r'^https://ec.europa.eu/eurostat/',url)]            
        
        if len(urls) > 0: 
            print()
            print(key,': ',value[0]/valid_urls_sum)
            sump += value[0]/valid_urls_sum
            for url in urls:
                print(url)
    #print('sump= ',sump)    
   
    
def query_build1(value):
    style = {'description_width': 'initial'}
    
    Keywords = widgets.Text(
        value=value,
        placeholder='Type something',
        description='Keywords:',
        disabled=False
  )

    Glossaries= widgets.RadioButtons(
        options=['All vocabularies','Eurostat', 'OECD'],
        description='Select:',
        value='OECD',
        disabled=False
  )

  #ui = widgets.HBox([Keywords])
    ui = widgets.HBox([Keywords,Glossaries])
    out = widgets.interactive_output(change_top_articles, {'Keywords': Keywords,'Glossaries' : Glossaries})
    display(ui, out)
    
query_build1(value='carbon dioxide greenhouse')  

<IPython.core.display.Javascript object>

Output()